In [0]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os
import numpy as np # linear algebra
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import warnings
# filter warnings
warnings.filterwarnings('ignore')
import os
from subprocess import check_output

# Import various componenets for model building
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop

# Import the backend
from keras import backend as K

import numpy as np
import cv2
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import glob
import os
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.cluster import KMeans

Using TensorFlow backend.


In [0]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"gailyn","key":"f838642402b2c5fc90e6d2d0e8d7b171"}'}

In [0]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 62 Oct 23 13:41 kaggle.json


In [0]:
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d moltean/fruits 

fruits.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!ls 

 fruits      'kaggle (1).json'	 kaggle.json
 fruits.zip  'kaggle (2).json'	 sample_data


In [0]:
import zipfile

In [0]:
with zipfile.ZipFile('fruits.zip', 'r') as zip_ref:
  zip_ref.extractall('fruits')

In [0]:
!ls fruits/fruits-360_dataset/fruits-360

LICENSE  papers  readme.md  Test  test-multiple_fruits	Training


In [0]:
train_path = 'fruits/fruits-360_dataset/fruits-360/Training'
test_path = 'fruits/fruits-360_dataset/fruits-360/Test'

In [0]:
# First, we are going to load the file names and their respective target labels into numpy array
from sklearn.datasets import load_files
import numpy as np

train_dir = 'fruits/fruits-360_dataset/fruits-360/Training'
test_dir = 'fruits/fruits-360_dataset/fruits-360/Test'

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np.array(data['target'])
    target_labels = np.array(data['target_names'])
    return files,targets,target_labels
    
x_train, y_train,target_labels = load_dataset(train_dir)
x_test, y_test,_ = load_dataset(test_dir)
print('Loading complete!')

print('Training set size : ' , x_train.shape[0])
print('Testing set size : ', x_test.shape[0])

Loading complete!
Training set size :  60498
Testing set size :  20622


In [0]:
# Let's confirm the number of classes 
no_of_classes = len(np.unique(y_train))
no_of_classes

120

In [0]:
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train,no_of_classes)
y_test = np_utils.to_categorical(y_test,no_of_classes)
y_train[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.], dtype=float32)

In [0]:
# Now, we have to divide the validation set into test and validation set
x_test,x_valid = x_test[7000:],x_test[:7000]
y_test,y_vaild = y_test[7000:],y_test[:7000]
print('Vaildation X : ',x_valid.shape)
print('Vaildation y :',y_vaild.shape)
print('Test X : ',x_test.shape)
print('Test y : ',y_test.shape)

Vaildation X :  (7000,)
Vaildation y : (7000, 120)
Test X :  (13622,)
Test y :  (13622, 120)


In [0]:
# We just have the file names in the x set. Let's load the images and convert them into array.
from keras.preprocessing.image import array_to_img, img_to_array, load_img

def convert_image_to_array(files):
    images_as_array=[]
    for file in files:
        # Convert to Numpy Array
        images_as_array.append(img_to_array(load_img(file)))
    return images_as_array

x_train = np.array(convert_image_to_array(x_train))
print('Training set shape : ',x_train.shape)

x_valid = np.array(convert_image_to_array(x_valid))
print('Validation set shape : ',x_valid.shape)

x_test = np.array(convert_image_to_array(x_test))
print('Test set shape : ',x_test.shape)

print('1st training image shape ',x_train[0].shape)

Training set shape :  (60498, 100, 100, 3)
Validation set shape :  (7000, 100, 100, 3)
Test set shape :  (13622, 100, 100, 3)
1st training image shape  (100, 100, 3)


In [0]:
# time to re-scale so that all the pixel values lie within 0 to 1
x_train = x_train.astype('float32')/255
x_valid = x_valid.astype('float32')/255
x_test = x_test.astype('float32')/255
x_train[0]

array([[[0.9764706 , 1.        , 0.99215686],
        [0.9843137 , 1.        , 0.99215686],
        [0.99607843, 1.        , 1.        ],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ]],

       [[0.9843137 , 1.        , 0.99215686],
        [0.9882353 , 1.        , 0.99215686],
        [0.99607843, 1.        , 1.        ],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ]],

       [[0.99607843, 1.        , 1.        ],
        [0.99607843, 1.        , 1.        ],
        [0.99607843, 1.        , 1.        ],
        ...,
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ]],

       ...,

       [[1.        , 1.        , 1.        ],
        [1.        , 1.        , 1.        ],
        [1.        , 1

In [0]:
# 3 Conv layers followed by maxpooling layers.
# At the end we add dropout, flatten and some fully connected layers

from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras import backend as K

model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = 2,input_shape=(100,100,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 32,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 64,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 128,kernel_size = 2,activation= 'relu',padding='same'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(150))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(120,activation = 'softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 100, 100, 16)      208       
_________________________________________________________________
activation_3 (Activation)    (None, 100, 100, 16)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 50, 50, 32)        2080      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 25, 25, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 25, 25, 64)        8256      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 64)       

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
print('Compiled!')

Compiled!


In [0]:
batch_size = 32


history = model.fit(x_train,y_train,
        batch_size = 32,
        epochs=15,
        validation_data=(x_valid, y_vaild),
        #callbacks = [checkpointer],
        verbose=2, shuffle=True)

Train on 60498 samples, validate on 7000 samples
Epoch 1/15
 - 254s - loss: 0.0574 - acc: 0.9849 - val_loss: 0.0725 - val_acc: 0.9819
Epoch 2/15
 - 254s - loss: 0.0560 - acc: 0.9861 - val_loss: 0.0688 - val_acc: 0.9873
Epoch 3/15
 - 257s - loss: 0.0533 - acc: 0.9873 - val_loss: 0.0769 - val_acc: 0.9841
Epoch 4/15
 - 255s - loss: 0.0624 - acc: 0.9861 - val_loss: 0.0642 - val_acc: 0.9870
Epoch 5/15
 - 255s - loss: 0.0648 - acc: 0.9869 - val_loss: 0.1237 - val_acc: 0.9769
Epoch 6/15
 - 254s - loss: 0.0636 - acc: 0.9871 - val_loss: 0.1348 - val_acc: 0.9827
Epoch 7/15
 - 256s - loss: 0.0696 - acc: 0.9874 - val_loss: 0.0708 - val_acc: 0.9904
Epoch 8/15
 - 256s - loss: 0.0729 - acc: 0.9875 - val_loss: 0.0550 - val_acc: 0.9879
Epoch 9/15
 - 255s - loss: 0.0703 - acc: 0.9877 - val_loss: 0.0627 - val_acc: 0.9901
Epoch 10/15
 - 256s - loss: 0.0704 - acc: 0.9884 - val_loss: 0.1240 - val_acc: 0.9867
Epoch 11/15
 - 259s - loss: 0.0834 - acc: 0.9874 - val_loss: 0.1494 - val_acc: 0.9847
Epoch 12/15
 -